In [24]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, classification_report
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from transformers import AdamW
from tqdm import tqdm

In [25]:
language="it"
modelname="ItalianBert"
input = "Dodiom"
test = "Dodiom"

In [26]:
if input == "gpt4":
  gpt4_flag=True
else:
  gpt4_flag=False

In [27]:
# JP encoding='euc-jp' other encoding normal CHECK ALWAYS DATASETS
df_train = pd.read_csv(f'dodiom_it_train.csv')
df_eval = pd.read_csv(f'dodiom_it_eval.csv')
df_test = pd.read_csv(f'dodiom_it_test.csv')

In [28]:
def get_category_mapping(language):
    if language in ['en', 'jp']:
        category_mapping = {
            'figurative': 'idiom',
            'literal': 'nonidiom'
        }
    elif language == 'tr':
        category_mapping = {
            'mecaz': 'idiom',
            'gerçek': 'nonidiom'
        }
    elif language == 'it':
        category_mapping = {
            'figurato': 'idiom',
            'letterale': 'nonidiom'
        }
    else:
        raise ValueError(f"Language '{language}' is not supported.")

    return category_mapping

In [29]:
category_mapping = get_category_mapping(language)
print(category_mapping)

{'figurato': 'idiom', 'letterale': 'nonidiom'}


In [30]:
if gpt4_flag or language in ['en', 'jp']:
  df_train['category'] = df_train['category'].map(category_mapping)
df_train

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,abbaiare alla luna,Non ti stanchi di abbaiare per qualsiasi cosa ...,idiom,0.333333,1,2,0,"[4, 8, 9]","['abbaiare', 'alla', 'luna']","['non', 'ti', 'stanco', 'di', 'abbaiare', 'per...","['Non', 'ti', 'stanchi', 'di', 'abbaiare', 'pe..."
1,abbaiare alla luna,Ma c’è ancora chi abbaia alla luna.,idiom,1.000000,3,0,0,"[5, 6, 7]","['abbaia', 'alla', 'luna']","['ma', ""c'"", 'essere', 'ancora', 'chi', 'abbai...","['Ma', 'c’', 'è', 'ancora', 'chi', 'abbaia', '..."
2,abbaiare alla luna,"Smettila di abbaiare alla luna, sei una lagna",idiom,1.000000,2,0,0,"[2, 3, 4]","['abbaiare', 'alla', 'luna']","['Smettila', 'di', 'abbaiare', 'alla', 'luna',...","['Smettila', 'di', 'abbaiare', 'alla', 'luna',..."
3,abbaiare alla luna,Trump si lamenta dei risultati delle ultime el...,idiom,0.000000,0,0,0,"[11, 12, 13]","['abbaiando', 'alla', 'luna']","['Trump', 'si', 'lamentare', 'del', 'risultato...","['Trump', 'si', 'lamenta', 'dei', 'risultati',..."
4,abbaiare alla luna,"è inutile che, dopo esserti reso conto dei tuo...",idiom,0.800000,4,1,0,"[12, 13, 14]","['abbai', 'alla', 'luna']","['essere', 'inutile', 'che', ',', 'dopo', 'ess...","['è', 'inutile', 'che', ',', 'dopo', 'esserti'..."
...,...,...,...,...,...,...,...,...,...,...,...
4455,voltare la faccia,devi voltare la faccia quando hai finito di pa...,nonidiom,0.500000,2,2,0,"[1, 3]","['voltare', 'faccia']","['deviare', 'voltare', 'la', 'faccia', 'quando...","['devi', 'voltare', 'la', 'faccia', 'quando', ..."
4456,voltare la faccia,Mi ha voltato la faccia quando mi ha vista,nonidiom,1.000000,5,0,0,"[2, 4]","['voltato', 'faccia']","['mi', 'avere', 'voltare', 'la', 'faccia', 'qu...","['Mi', 'ha', 'voltato', 'la', 'faccia', 'quand..."
4457,voltare la faccia,"non ti ho voltato di proposito la faccia, non ...",nonidiom,1.000000,5,0,0,"[3, 7]","['voltato', 'faccia']","['non', 'ti', 'avere', 'voltare', 'di', 'propo...","['non', 'ti', 'ho', 'voltato', 'di', 'proposit..."
4458,voltare la faccia,Non puoi voltare la faccia alla professoressa,nonidiom,0.600000,3,1,1,"[2, 4]","['voltare', 'faccia']","['non', 'potere', 'voltare', 'la', 'faccia', '...","['Non', 'puoi', 'voltare', 'la', 'faccia', 'al..."


In [31]:
if gpt4_flag or language in ['en', 'jp']:
  df_eval['category'] = df_eval['category'].map(category_mapping)
df_eval

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,abbaiare alla luna,Alla luna abbaiano solo i cani quindi prova a ...,idiom,0.500000,1,1,0,"[2, 0, 1]","['abbaiano', 'Alla', 'luna']","['alla', 'luna', 'abbaiare', 'solo', 'il', 'ca...","['Alla', 'luna', 'abbaiano', 'solo', 'i', 'can..."
1,abbaiare alla luna,Parlare con Mario è come abbaiare alla luna.,idiom,1.000000,3,0,0,"[5, 6, 7]","['abbaiare', 'alla', 'luna']","['parlare', 'con', 'Mario', 'essere', 'come', ...","['Parlare', 'con', 'Mario', 'è', 'come', 'abba..."
2,abbaiare alla luna,Il suo comportamento mi fa venire voglia di ab...,idiom,0.500000,3,3,0,"[8, 9, 10]","['abbaiare', 'alla', 'luna']","['il', 'suo', 'comportamento', 'mi', 'fare', '...","['Il', 'suo', 'comportamento', 'mi', 'fa', 've..."
3,abbaiare alla luna,Mi hai stufato tu e il tuo continuo abbaiare a...,idiom,1.000000,2,0,0,"[8, 9, 10]","['abbaiare', 'alla', 'luna']","['mi', 'avere', 'stufato', 'tu', 'e', 'il', 't...","['Mi', 'hai', 'stufato', 'tu', 'e', 'il', 'tuo..."
4,abbaiare alla luna,Abbaia alla luna più che puoi,idiom,1.000000,2,0,0,"[0, 1, 2]","['Abbaia', 'alla', 'luna']","['abbaiare', 'alla', 'luna', 'più', 'che', 'po...","['Abbaia', 'alla', 'luna', 'più', 'che', 'puoi']"
...,...,...,...,...,...,...,...,...,...,...,...
784,voltare la faccia,"Mia sorella quando parliamo, continua a voltar...",nonidiom,0.666667,4,2,0,"[7, 9]","['voltare', 'faccia']","['Mia', 'sorella', 'quando', 'parlare', ',', '...","['Mia', 'sorella', 'quando', 'parliamo', ',', ..."
785,voltare la faccia,Ha voltato la faccia quando gli ho chiesto di ...,nonidiom,0.200000,1,4,0,"[1, 3]","['voltato', 'faccia']","['avere', 'voltare', 'la', 'faccia', 'quando',...","['Ha', 'voltato', 'la', 'faccia', 'quando', 'g..."
786,voltare la faccia,Voltò la faccia verso di me per un ultimo sorr...,nonidiom,1.000000,1,0,0,"[0, 2]","['Voltò', 'faccia']","['voltare', 'la', 'faccia', 'verso', 'di', 'me...","['Voltò', 'la', 'faccia', 'verso', 'di', 'me',..."
787,voltare la faccia,Hai voltato la faccia proprio mentre ti stavo ...,nonidiom,1.000000,2,0,0,"[1, 3]","['voltato', 'faccia']","['avere', 'voltare', 'la', 'faccia', 'proprio'...","['Hai', 'voltato', 'la', 'faccia', 'proprio', ..."


In [32]:
if language in ['en', 'jp']:
  df_test['category'] = df_test['category'].map(category_mapping)

In [33]:
df_test

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,abbaiare alla luna,Abbaiare alla luna o roteare gatti non può cur...,idiom,1.000000,2,0,0,"[0, 1, 2]","['Abbaiare', 'alla', 'luna']","['abbaiare', 'alla', 'luna', 'o', 'roteare', '...","['Abbaiare', 'alla', 'luna', 'o', 'roteare', '..."
1,abbaiare alla luna,Non è possibile che tu debba abbaiare per tutt...,idiom,0.500000,1,1,0,"[6, 9, 10]","['abbaiare', 'alla', 'luna']","['non', 'essere', 'possibile', 'che', 'tu', 'd...","['Non', 'è', 'possibile', 'che', 'tu', 'debba'..."
2,abbaiare alla luna,Chiedere alla prof di pubblicare le slide è co...,idiom,0.500000,1,1,0,"[9, 10, 11]","['abbaiare', 'alla', 'luna']","['chiedere', 'alla', 'prof', 'di', 'pubblicare...","['Chiedere', 'alla', 'prof', 'di', 'pubblicare..."
3,abbaiare alla luna,Abbai almeno 10 volte al giorno alla luna,idiom,0.000000,0,2,0,"[0, 6, 7]","['Abbai', 'alla', 'luna']","['abbaiare', 'almeno', '10', 'volta', 'al', 'g...","['Abbai', 'almeno', '10', 'volte', 'al', 'gior..."
4,abbaiare alla luna,Sprechi troppo tempo ad abbaiare alla luna,idiom,0.666667,2,1,0,"[4, 5, 6]","['abbaiare', 'alla', 'luna']","['sprecare', 'troppo', 'tempo', 'ad', 'abbaiar...","['Sprechi', 'troppo', 'tempo', 'ad', 'abbaiare..."
...,...,...,...,...,...,...,...,...,...,...,...
2279,voltare la faccia,"Puoi voltare la faccia per favore, voglio fart...",nonidiom,1.000000,3,0,0,"[1, 3]","['voltare', 'faccia']","['potere', 'voltare', 'la', 'faccia', 'per', '...","['Puoi', 'voltare', 'la', 'faccia', 'per', 'fa..."
2280,voltare la faccia,per favore puoi voltare la faccia verso di me ...,nonidiom,1.000000,3,0,0,"[3, 5]","['voltare', 'faccia']","['per', 'favore', 'potere', 'voltare', 'la', '...","['per', 'favore', 'puoi', 'voltare', 'la', 'fa..."
2281,voltare la faccia,devi voltare la faccia da entrambi i lati per ...,nonidiom,1.000000,3,0,0,"[1, 3]","['voltare', 'faccia']","['deviare', 'voltare', 'la', 'faccia', 'da', '...","['devi', 'voltare', 'la', 'faccia', 'da', 'ent..."
2282,voltare la faccia,Appena mi ha visto ha voltato la faccia,nonidiom,0.500000,1,1,0,"[5, 7]","['voltato', 'faccia']","['appena', 'mi', 'avere', 'vistare', 'avere', ...","['Appena', 'mi', 'ha', 'visto', 'ha', 'voltato..."


In [34]:
df_train['category'] = df_train['category'].str.lower()
df_eval['category'] = df_eval['category'].str.lower()
df_test['category'] = df_test['category'].str.lower()

In [35]:
df_train.head()

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,abbaiare alla luna,Non ti stanchi di abbaiare per qualsiasi cosa ...,idiom,0.333333,1,2,0,"[4, 8, 9]","['abbaiare', 'alla', 'luna']","['non', 'ti', 'stanco', 'di', 'abbaiare', 'per...","['Non', 'ti', 'stanchi', 'di', 'abbaiare', 'pe..."
1,abbaiare alla luna,Ma c’è ancora chi abbaia alla luna.,idiom,1.000000,3,0,0,"[5, 6, 7]","['abbaia', 'alla', 'luna']","['ma', ""c'"", 'essere', 'ancora', 'chi', 'abbai...","['Ma', 'c’', 'è', 'ancora', 'chi', 'abbaia', '..."
2,abbaiare alla luna,"Smettila di abbaiare alla luna, sei una lagna",idiom,1.000000,2,0,0,"[2, 3, 4]","['abbaiare', 'alla', 'luna']","['Smettila', 'di', 'abbaiare', 'alla', 'luna',...","['Smettila', 'di', 'abbaiare', 'alla', 'luna',..."
3,abbaiare alla luna,Trump si lamenta dei risultati delle ultime el...,idiom,0.000000,0,0,0,"[11, 12, 13]","['abbaiando', 'alla', 'luna']","['Trump', 'si', 'lamentare', 'del', 'risultato...","['Trump', 'si', 'lamenta', 'dei', 'risultati',..."
4,abbaiare alla luna,"è inutile che, dopo esserti reso conto dei tuo...",idiom,0.800000,4,1,0,"[12, 13, 14]","['abbai', 'alla', 'luna']","['essere', 'inutile', 'che', ',', 'dopo', 'ess...","['è', 'inutile', 'che', ',', 'dopo', 'esserti'..."


In [36]:
df_eval.head()

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,abbaiare alla luna,Alla luna abbaiano solo i cani quindi prova a ...,idiom,0.5,1,1,0,"[2, 0, 1]","['abbaiano', 'Alla', 'luna']","['alla', 'luna', 'abbaiare', 'solo', 'il', 'ca...","['Alla', 'luna', 'abbaiano', 'solo', 'i', 'can..."
1,abbaiare alla luna,Parlare con Mario è come abbaiare alla luna.,idiom,1.0,3,0,0,"[5, 6, 7]","['abbaiare', 'alla', 'luna']","['parlare', 'con', 'Mario', 'essere', 'come', ...","['Parlare', 'con', 'Mario', 'è', 'come', 'abba..."
2,abbaiare alla luna,Il suo comportamento mi fa venire voglia di ab...,idiom,0.5,3,3,0,"[8, 9, 10]","['abbaiare', 'alla', 'luna']","['il', 'suo', 'comportamento', 'mi', 'fare', '...","['Il', 'suo', 'comportamento', 'mi', 'fa', 've..."
3,abbaiare alla luna,Mi hai stufato tu e il tuo continuo abbaiare a...,idiom,1.0,2,0,0,"[8, 9, 10]","['abbaiare', 'alla', 'luna']","['mi', 'avere', 'stufato', 'tu', 'e', 'il', 't...","['Mi', 'hai', 'stufato', 'tu', 'e', 'il', 'tuo..."
4,abbaiare alla luna,Abbaia alla luna più che puoi,idiom,1.0,2,0,0,"[0, 1, 2]","['Abbaia', 'alla', 'luna']","['abbaiare', 'alla', 'luna', 'più', 'che', 'po...","['Abbaia', 'alla', 'luna', 'più', 'che', 'puoi']"


In [37]:
df_test.head()

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,abbaiare alla luna,Abbaiare alla luna o roteare gatti non può cur...,idiom,1.000000,2,0,0,"[0, 1, 2]","['Abbaiare', 'alla', 'luna']","['abbaiare', 'alla', 'luna', 'o', 'roteare', '...","['Abbaiare', 'alla', 'luna', 'o', 'roteare', '..."
1,abbaiare alla luna,Non è possibile che tu debba abbaiare per tutt...,idiom,0.500000,1,1,0,"[6, 9, 10]","['abbaiare', 'alla', 'luna']","['non', 'essere', 'possibile', 'che', 'tu', 'd...","['Non', 'è', 'possibile', 'che', 'tu', 'debba'..."
2,abbaiare alla luna,Chiedere alla prof di pubblicare le slide è co...,idiom,0.500000,1,1,0,"[9, 10, 11]","['abbaiare', 'alla', 'luna']","['chiedere', 'alla', 'prof', 'di', 'pubblicare...","['Chiedere', 'alla', 'prof', 'di', 'pubblicare..."
3,abbaiare alla luna,Abbai almeno 10 volte al giorno alla luna,idiom,0.000000,0,2,0,"[0, 6, 7]","['Abbai', 'alla', 'luna']","['abbaiare', 'almeno', '10', 'volta', 'al', 'g...","['Abbai', 'almeno', '10', 'volte', 'al', 'gior..."
4,abbaiare alla luna,Sprechi troppo tempo ad abbaiare alla luna,idiom,0.666667,2,1,0,"[4, 5, 6]","['abbaiare', 'alla', 'luna']","['sprecare', 'troppo', 'tempo', 'ad', 'abbaiar...","['Sprechi', 'troppo', 'tempo', 'ad', 'abbaiare..."


In [38]:
# Encode the 'category' column
label_encoder = LabelEncoder()
df_train['category_encoded'] = label_encoder.fit_transform(df_train['category'])
df_eval['category_encoded'] = label_encoder.transform(df_eval['category'])
df_test['category_encoded'] = label_encoder.transform(df_test['category'])

In [39]:
df_train

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words,category_encoded
0,abbaiare alla luna,Non ti stanchi di abbaiare per qualsiasi cosa ...,idiom,0.333333,1,2,0,"[4, 8, 9]","['abbaiare', 'alla', 'luna']","['non', 'ti', 'stanco', 'di', 'abbaiare', 'per...","['Non', 'ti', 'stanchi', 'di', 'abbaiare', 'pe...",0
1,abbaiare alla luna,Ma c’è ancora chi abbaia alla luna.,idiom,1.000000,3,0,0,"[5, 6, 7]","['abbaia', 'alla', 'luna']","['ma', ""c'"", 'essere', 'ancora', 'chi', 'abbai...","['Ma', 'c’', 'è', 'ancora', 'chi', 'abbaia', '...",0
2,abbaiare alla luna,"Smettila di abbaiare alla luna, sei una lagna",idiom,1.000000,2,0,0,"[2, 3, 4]","['abbaiare', 'alla', 'luna']","['Smettila', 'di', 'abbaiare', 'alla', 'luna',...","['Smettila', 'di', 'abbaiare', 'alla', 'luna',...",0
3,abbaiare alla luna,Trump si lamenta dei risultati delle ultime el...,idiom,0.000000,0,0,0,"[11, 12, 13]","['abbaiando', 'alla', 'luna']","['Trump', 'si', 'lamentare', 'del', 'risultato...","['Trump', 'si', 'lamenta', 'dei', 'risultati',...",0
4,abbaiare alla luna,"è inutile che, dopo esserti reso conto dei tuo...",idiom,0.800000,4,1,0,"[12, 13, 14]","['abbai', 'alla', 'luna']","['essere', 'inutile', 'che', ',', 'dopo', 'ess...","['è', 'inutile', 'che', ',', 'dopo', 'esserti'...",0
...,...,...,...,...,...,...,...,...,...,...,...,...
4455,voltare la faccia,devi voltare la faccia quando hai finito di pa...,nonidiom,0.500000,2,2,0,"[1, 3]","['voltare', 'faccia']","['deviare', 'voltare', 'la', 'faccia', 'quando...","['devi', 'voltare', 'la', 'faccia', 'quando', ...",1
4456,voltare la faccia,Mi ha voltato la faccia quando mi ha vista,nonidiom,1.000000,5,0,0,"[2, 4]","['voltato', 'faccia']","['mi', 'avere', 'voltare', 'la', 'faccia', 'qu...","['Mi', 'ha', 'voltato', 'la', 'faccia', 'quand...",1
4457,voltare la faccia,"non ti ho voltato di proposito la faccia, non ...",nonidiom,1.000000,5,0,0,"[3, 7]","['voltato', 'faccia']","['non', 'ti', 'avere', 'voltare', 'di', 'propo...","['non', 'ti', 'ho', 'voltato', 'di', 'proposit...",1
4458,voltare la faccia,Non puoi voltare la faccia alla professoressa,nonidiom,0.600000,3,1,1,"[2, 4]","['voltare', 'faccia']","['non', 'potere', 'voltare', 'la', 'faccia', '...","['Non', 'puoi', 'voltare', 'la', 'faccia', 'al...",1


In [40]:
df_eval

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words,category_encoded
0,abbaiare alla luna,Alla luna abbaiano solo i cani quindi prova a ...,idiom,0.500000,1,1,0,"[2, 0, 1]","['abbaiano', 'Alla', 'luna']","['alla', 'luna', 'abbaiare', 'solo', 'il', 'ca...","['Alla', 'luna', 'abbaiano', 'solo', 'i', 'can...",0
1,abbaiare alla luna,Parlare con Mario è come abbaiare alla luna.,idiom,1.000000,3,0,0,"[5, 6, 7]","['abbaiare', 'alla', 'luna']","['parlare', 'con', 'Mario', 'essere', 'come', ...","['Parlare', 'con', 'Mario', 'è', 'come', 'abba...",0
2,abbaiare alla luna,Il suo comportamento mi fa venire voglia di ab...,idiom,0.500000,3,3,0,"[8, 9, 10]","['abbaiare', 'alla', 'luna']","['il', 'suo', 'comportamento', 'mi', 'fare', '...","['Il', 'suo', 'comportamento', 'mi', 'fa', 've...",0
3,abbaiare alla luna,Mi hai stufato tu e il tuo continuo abbaiare a...,idiom,1.000000,2,0,0,"[8, 9, 10]","['abbaiare', 'alla', 'luna']","['mi', 'avere', 'stufato', 'tu', 'e', 'il', 't...","['Mi', 'hai', 'stufato', 'tu', 'e', 'il', 'tuo...",0
4,abbaiare alla luna,Abbaia alla luna più che puoi,idiom,1.000000,2,0,0,"[0, 1, 2]","['Abbaia', 'alla', 'luna']","['abbaiare', 'alla', 'luna', 'più', 'che', 'po...","['Abbaia', 'alla', 'luna', 'più', 'che', 'puoi']",0
...,...,...,...,...,...,...,...,...,...,...,...,...
784,voltare la faccia,"Mia sorella quando parliamo, continua a voltar...",nonidiom,0.666667,4,2,0,"[7, 9]","['voltare', 'faccia']","['Mia', 'sorella', 'quando', 'parlare', ',', '...","['Mia', 'sorella', 'quando', 'parliamo', ',', ...",1
785,voltare la faccia,Ha voltato la faccia quando gli ho chiesto di ...,nonidiom,0.200000,1,4,0,"[1, 3]","['voltato', 'faccia']","['avere', 'voltare', 'la', 'faccia', 'quando',...","['Ha', 'voltato', 'la', 'faccia', 'quando', 'g...",1
786,voltare la faccia,Voltò la faccia verso di me per un ultimo sorr...,nonidiom,1.000000,1,0,0,"[0, 2]","['Voltò', 'faccia']","['voltare', 'la', 'faccia', 'verso', 'di', 'me...","['Voltò', 'la', 'faccia', 'verso', 'di', 'me',...",1
787,voltare la faccia,Hai voltato la faccia proprio mentre ti stavo ...,nonidiom,1.000000,2,0,0,"[1, 3]","['voltato', 'faccia']","['avere', 'voltare', 'la', 'faccia', 'proprio'...","['Hai', 'voltato', 'la', 'faccia', 'proprio', ...",1


In [41]:
df_test

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words,category_encoded
0,abbaiare alla luna,Abbaiare alla luna o roteare gatti non può cur...,idiom,1.000000,2,0,0,"[0, 1, 2]","['Abbaiare', 'alla', 'luna']","['abbaiare', 'alla', 'luna', 'o', 'roteare', '...","['Abbaiare', 'alla', 'luna', 'o', 'roteare', '...",0
1,abbaiare alla luna,Non è possibile che tu debba abbaiare per tutt...,idiom,0.500000,1,1,0,"[6, 9, 10]","['abbaiare', 'alla', 'luna']","['non', 'essere', 'possibile', 'che', 'tu', 'd...","['Non', 'è', 'possibile', 'che', 'tu', 'debba'...",0
2,abbaiare alla luna,Chiedere alla prof di pubblicare le slide è co...,idiom,0.500000,1,1,0,"[9, 10, 11]","['abbaiare', 'alla', 'luna']","['chiedere', 'alla', 'prof', 'di', 'pubblicare...","['Chiedere', 'alla', 'prof', 'di', 'pubblicare...",0
3,abbaiare alla luna,Abbai almeno 10 volte al giorno alla luna,idiom,0.000000,0,2,0,"[0, 6, 7]","['Abbai', 'alla', 'luna']","['abbaiare', 'almeno', '10', 'volta', 'al', 'g...","['Abbai', 'almeno', '10', 'volte', 'al', 'gior...",0
4,abbaiare alla luna,Sprechi troppo tempo ad abbaiare alla luna,idiom,0.666667,2,1,0,"[4, 5, 6]","['abbaiare', 'alla', 'luna']","['sprecare', 'troppo', 'tempo', 'ad', 'abbaiar...","['Sprechi', 'troppo', 'tempo', 'ad', 'abbaiare...",0
...,...,...,...,...,...,...,...,...,...,...,...,...
2279,voltare la faccia,"Puoi voltare la faccia per favore, voglio fart...",nonidiom,1.000000,3,0,0,"[1, 3]","['voltare', 'faccia']","['potere', 'voltare', 'la', 'faccia', 'per', '...","['Puoi', 'voltare', 'la', 'faccia', 'per', 'fa...",1
2280,voltare la faccia,per favore puoi voltare la faccia verso di me ...,nonidiom,1.000000,3,0,0,"[3, 5]","['voltare', 'faccia']","['per', 'favore', 'potere', 'voltare', 'la', '...","['per', 'favore', 'puoi', 'voltare', 'la', 'fa...",1
2281,voltare la faccia,devi voltare la faccia da entrambi i lati per ...,nonidiom,1.000000,3,0,0,"[1, 3]","['voltare', 'faccia']","['deviare', 'voltare', 'la', 'faccia', 'da', '...","['devi', 'voltare', 'la', 'faccia', 'da', 'ent...",1
2282,voltare la faccia,Appena mi ha visto ha voltato la faccia,nonidiom,0.500000,1,1,0,"[5, 7]","['voltato', 'faccia']","['appena', 'mi', 'avere', 'vistare', 'avere', ...","['Appena', 'mi', 'ha', 'visto', 'ha', 'voltato...",1


In [42]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [43]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-italian-cased")

# Tokenize the data
train_encodings = tokenizer(list(df_train['submission']), truncation=True, padding=True, return_tensors="pt")
val_encodings = tokenizer(list(df_eval['submission']), truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(list(df_test['submission']), truncation=True, padding=True, return_tensors="pt")
# Convert to torch tensors
train_labels = torch.tensor(df_train['category_encoded'].values)
val_labels = torch.tensor(df_eval['category_encoded'].values)
test_labels = torch.tensor(df_test['category_encoded'].values)

# Prepare to datasets
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)
test_dataset = TextDataset(test_encodings, test_labels)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [44]:
num_labels = len(label_encoder.classes_)
model = AutoModelForSequenceClassification.from_pretrained("dbmdz/bert-base-italian-cased", num_labels=num_labels)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-6)

epochs = 4
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Average loss across all batches
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss}")

    # Evaluation step
    model.eval()
    total_eval_accuracy = 0
    for batch in tqdm(val_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        accuracy = (predictions == batch['labels']).cpu().numpy().mean() * 100
        total_eval_accuracy += accuracy

    avg_val_accuracy = total_eval_accuracy / len(val_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Validation Accuracy: {avg_val_accuracy:.2f}%")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-italian-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/558 [00:00<?, ?it/s]<ipython-input-42-845b27163e62>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-42-845b27163e62>:8: Use

Epoch 1/4 | Train Loss: 0.4846729432703346


100%|██████████| 99/99 [00:01<00:00, 61.83it/s]


Epoch 1/4 | Validation Accuracy: 84.62%


100%|██████████| 558/558 [00:56<00:00,  9.91it/s]


...

Epoch 3/4 | Validation Accuracy: 88.06%


100%|██████████| 558/558 [00:56<00:00,  9.91it/s]


Epoch 4/4 | Train Loss: 0.18487328357366997


100%|██████████| 99/99 [00:01<00:00, 63.19it/s]

Epoch 4/4 | Validation Accuracy: 88.99%


In [45]:
# Ensure the model is in evaluation mode
model.eval()

# Move the model to the appropriate device
model.to(device)

# Initialize variables to track accuracy
total_correct = 0
total_predictions = 0

# Store true labels and predictions for F1 score calculation
all_labels = []
all_predictions = []

# No gradient updates needed for evaluation
with torch.no_grad():
    for batch in test_loader:
        # Move batch to the same device as the model
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Get predictions
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        # Update tracking variables
        total_correct += (predictions == batch['labels']).sum().item()
        total_predictions += predictions.size(0)

        # Store predictions and true labels
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

# Calculate accuracy
test_accuracy = total_correct / total_predictions
print(f'Test Accuracy: {test_accuracy:.4f}')

# Calculate F1 scores
f1_weighted = f1_score(all_labels, all_predictions, average='weighted')
f1_macro = f1_score(all_labels, all_predictions, average='macro')

print(f'Weighted F1 Score: {f1_weighted:.4f}')
print(f'Macro F1 Score: {f1_macro:.4f}')


<ipython-input-42-845b27163e62>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-42-845b27163e62>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Test Accuracy: 0.8975
Weighted F1 Score: 0.8969
Macro F1 Score: 0.8929


In [46]:
# Generate classification report
report = classification_report(all_labels, all_predictions, digits=4)
print(f'Classification Report for {modelname}, language {language.upper()}, Train {input.upper()}, Test {test.upper()}:')
print(report)

Classification Report for ItalianBert, language IT, Train DODIOM, Test DODIOM:
              precision    recall  f1-score   support

           0     0.8926    0.9390    0.9152      1345
           1     0.9056    0.8381    0.8706       939

    accuracy                         0.8975      2284
   macro avg     0.8991    0.8886    0.8929      2284
weighted avg     0.8979    0.8975    0.8969      2284

